In [25]:
# importamos las librerías con las que vamos a trabajar

# Trabajar con bases de datos y python
# -----------------------------------------------------------------------
import psycopg2
from psycopg2 import OperationalError, errorcodes, errors


# Trabajar con DataFrames
# -----------------------------------------------------------------------
import pandas as pd

OKAY, necesito una bd, pero para ello primero lo debo transformar todo a dataframe

##### Convierto a DataFrame los muicipios, y los meto en una lista de tuplas para facilitar la insercción a la base de datos

In [38]:
df_municipios_raw = pd.read_csv("data/municipios.csv", index_col=0)
df_municipios = df_municipios_raw.T.reset_index().drop(columns=2)
df_municipios.columns = ["codigo_municipio","longitud","latitud"]
print(df_municipios.dtypes)
df_municipios.head(2)
df_municipios.to_csv("data/datos_municipios_lucas.csv")

codigo_municipio    object
longitud            object
latitud             object
dtype: object


In [27]:
lista_tuplas_municipios = []
for fila in df_municipios.values:
    lista_tuplas_municipios.append(tuple(fila))
lista_tuplas_municipios[0]

('acebeda-la', '41.0869583', '-3.624399')

##### Convierto a DataFrame los datos de aemet y los meto en una lista de tuplas para facilitar su insercción a la base de datos

In [28]:
df_aemet = pd.read_csv("data/df_aemet_final.csv")
print(df_aemet.dtypes)
# df_aemet.astype({"fecha": "datetime"})
df_aemet.drop("avisos", axis=1, inplace=True)
df_aemet.drop("prob._precip.__(%)", axis=1, inplace=True)
df_aemet['localizacion_id'] = df_aemet['localizacion_id'].str.replace(r'-\w{2}\d{5}', '', regex = True)
df_aemet.head(1)

fecha                     object
cielo                     object
temp._(°c)                 int64
sen._térmica_(°c)          int64
racha_máx._(km/h)        float64
precipitación_(mm)       float64
nieve_(mm)                 int64
humedad_relativa_(%)       int64
prob._precip.__(%)        object
prob._de_nieve_(%)         int64
prob._de_tormenta_(%)      int64
avisos                    object
dirección_viento          object
velocidad_del_viento       int64
localizacion_id           object
dtype: object


,fecha,cielo,temp._(°c),sen._térmica_(°c),racha_máx._(km/h),precipitación_(mm),nieve_(mm),humedad_relativa_(%),prob._de_nieve_(%),prob._de_tormenta_(%),dirección_viento,velocidad_del_viento,localizacion_id
0,2024-08-29 12:00:00,Nubes altas,21,21,25.0,0.0,0,69,0,75,S,7,acebeda-la-id28001


In [29]:
lista_tuplas_aemet = []
for fila in df_aemet.values:
    lista_tuplas_aemet.append(tuple(fila))
lista_tuplas_aemet[0]

('2024-08-29 12:00:00',
 'Nubes altas',
 21,
 21,
 25.0,
 0.0,
 0,
 69,
 0,
 75,
 'S',
 7,
 'acebeda-la-id28001')

##### Convierto los datos de foursquare a dataframe y preparo una lista de tuplas para facilitar la inserción en la base de datos

In [30]:
df_forsquere = pd.read_csv("data/api_foursquare.csv", index_col=0)
print(df_forsquere.dtypes)
df_forsquere.head(2)

municipio         object
category          object
fsq_id            object
closed_bucket     object
distance         float64
link              object
name              object
address           object
latitude         float64
longitude        float64
dtype: object


,municipio,category,fsq_id,closed_bucket,distance,link,name,address,latitude,longitude
0,ajalvir,Park,4d3c6c5b84d46ea87dddfd5c,LikelyOpen,563.0,/v3/places/4d3c6c5b84d46ea87dddfd5c,Diverjungla,"Calle Segovia, 20 (Pol. Ind. Donada), 28864 Aj...",40.529921,-3.484341
1,ajalvir,Park,4f54f7bfe4b036244d02685b,VeryLikelyOpen,1724.0,/v3/places/4f54f7bfe4b036244d02685b,Parque Infantil,NaN,40.543867,-3.464773


In [31]:
listas_tuplas_foursquare = []
for fila in df_forsquere.values:
    listas_tuplas_foursquare.append(tuple(fila))
listas_tuplas_foursquare[0]

('ajalvir',
 'Park',
 '4d3c6c5b84d46ea87dddfd5c',
 'LikelyOpen',
 563.0,
 '/v3/places/4d3c6c5b84d46ea87dddfd5c',
 'Diverjungla',
 'Calle Segovia, 20 (Pol. Ind. Donada), 28864 Ajalvir Comunidad de Madrid',
 40.529921,
 -3.484341)

## Conexión y creacción de tablas

In [32]:
try:
    conn = psycopg2.connect(
        database = "laboratorio",
        user = "postgres",
        password = "admin",
        host = "localhost",
        port = "5432"
    )
except OperationalError as e:
    if e.pgcode == errorcodes.INVALID_PASSWORD:
        print("contraseña incorrecta")
    elif e.pgcode == errorcodes.CONNECTION_EXCEPTION:
        print("Error de conexion")
    else:
        print(f"Ocurrio el error {e}")

cursor = conn.cursor()
query_creacion = """
    CREATE TABLE IF NOT EXISTS MUNICIPIOS (
    codigo_municipio VARCHAR(100) PRIMARY KEY,
    longitud VARCHAR(100),
    latitud VARCHAR(100)
);

CREATE TABLE IF NOT EXISTS AEMET (
    fecha VARCHAR(100),
    cielo VARCHAR(100),
    temp_°C INT,
    sen_térmica_°C INT,
    racha_máx_km_h DECIMAL,
    precipitación_mm DECIMAL,
    nieve_mm INT,
    humedad_relativa_ INT,
    prob_de_nieve_ INT,
    prob_de_tormenta_ INT,
    dirección_viento VARCHAR(100),
    velocidad_del_viento INT,
    localizacion_id VARCHAR(100) primary key,
    FOREIGN KEY (localizacion_id) REFERENCES MUNICIPIOS(codigo_municipio)
);

CREATE TABLE IF NOT EXISTS FOURSQUARE (
    municipio VARCHAR(100),
    category VARCHAR(100),
    fsq_id VARCHAR(100) primary key,
    closed_bucket VARCHAR(100),
    distance DECIMAL(10, 2),
    link VARCHAR(100),
    name VARCHAR(100),
    address VARCHAR(100),
    latitude DECIMAL(10, 6),
    longitude DECIMAL(10, 6),
    FOREIGN KEY (municipio) REFERENCES MUNICIPIOS(codigo_municipio)
);
"""


cursor.execute(query_creacion)
conn.commit()

In [33]:
query12 = """
drop table AEMET
"""
cursor.execute(query12)
conn.commit()

In [34]:
df_municipios

,codigo_municipio,longitud,latitud
0,acebeda-la,41.0869583,-3.624399
1,ajalvir,40.5342302,-3.4807818
2,alameda-del-valle,40.9177178,-3.8438216
3,alamo-el,40.2307296,-3.9905893
4,alcala-de-henares,40.4818396,-3.3644973
...,...,...,...
171,villar-del-olmo,40.3364465,-3.2355624
172,villarejo-de-salvanes,40.1683582,-3.2738764
173,villaviciosa-de-odon,40.3573787,-3.9002334
174,villavieja-del-lozoya,41.0063622,-3.6716291


## INSERCCION DE DATOS

##### Insercion de los datos df_municipios into la tabla municipio de mi base de datos

In [35]:
# query_inserción = 'insert into municipios (codigo_municipio, longitud, latitud) values (%s, %s, %s)'
# cursor.executemany(query_inserción, lista_tuplas_municipios)
# conn.commit()

##### Insercion de los datos de df_aemet en la tabla AEMET de mi base de datos

In [36]:
df_aemet.dtypes

fecha                     object
cielo                     object
temp._(°c)                 int64
sen._térmica_(°c)          int64
racha_máx._(km/h)        float64
precipitación_(mm)       float64
nieve_(mm)                 int64
humedad_relativa_(%)       int64
prob._de_nieve_(%)         int64
prob._de_tormenta_(%)      int64
dirección_viento          object
velocidad_del_viento       int64
localizacion_id           object
dtype: object

In [39]:
query_inserción = 'insert into AEMET (fecha, cielo, temp_°C, sen_térmica_°C, racha_máx_km_h, precipitación_mm, nieve_mm, humedad_relativa_, prob_de_nieve_, prob_de_tormenta_, dirección_viento, velocidad_del_viento, localizacion_id) values (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)'
cursor.executemany(query_inserción, lista_tuplas_aemet)
conn.commit()

InFailedSqlTransaction: transacción abortada, las órdenes serán ignoradas hasta el fin de bloque de transacción


In [ ]:
query_inserción = 'insert into FOURSQUARE (municipio, category, fsq_id, closed_bucket, distance, link, name, address, latitude, longitude) values (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)'
cursor.executemany(query_inserción, listas_tuplas_foursquare)
conn.commit()

InFailedSqlTransaction: transacción abortada, las órdenes serán ignoradas hasta el fin de bloque de transacción


In [ ]:
    municipio VARCHAR(100),
    category VARCHAR(100),
    fsq_id VARCHAR(100) primary key,
    closed_bucket VARCHAR(100),
    distance DECIMAL(10, 2),
    link VARCHAR(100),
    name VARCHAR(100),
    address VARCHAR(100),
    latitude DECIMAL(10, 6),
    longitude DECIMAL(10, 6),
    FOREIGN KEY (municipio) REFERENCES MUNICIPIOS(codigo_municipio)